In [5]:
import numpy as np
import math
from Crypto.Util.number import getPrime, inverse, bytes_to_long, long_to_bytes, GCD
import random

In [6]:
def isqrt(n):
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

In [7]:
def encrypt_rsa(m, e, N):
    assert m < N
    c = pow(m, e, N)
    return c

# Prerequisites

1. RSA + its prerequisites
2. Continued Fractions https://en.wikipedia.org/wiki/Continued_fraction

Variant:
1. Diophantine equations
* https://en.wikipedia.org/wiki/Diophantine_approximation
2. meet-in-the-middle

https://sagi.io/2016/04/crypto-classics-wieners-rsa-attack/

# Theory

**Task:** Break RSA if the decryption exponent $d$ is sufficiently small

## Continued fractions

Explanation: https://www.youtube.com/watch?v=hdozMSh9rOw&list=PLKXdxQAT3tCssgaWOy5vKXAR4WTPpRVYK&index=67

Let $\alpha_1 \in \mathbb{R}$ Then we have the sequence:  
$\alpha_1= a_1 + \cfrac 1 \alpha_2$ where $a_1 = \lfloor\alpha_1\rfloor$ 

$\alpha_2= a_2 + \cfrac 1 \alpha_3$ where $a_2 = \lfloor\alpha_2\rfloor$ 

$\alpha_3= a_3 + \cfrac 1 \alpha_4$ where $a_3 = \lfloor\alpha_3\rfloor$ 

Then $[a_1, a_2 ...]$ = continued fraction expansion of $\alpha_1$

So for $x \in \mathbb{R}$:

$$ x = a_0 + \cfrac 1 {a_1 + \cfrac 1 {a_2 + \dots}} => x = [a_0, a_1, ...]$$

*Intuition*: Get out the wholes in numbers

**Definition**  
Convergent = truncate the first $k$ terms => We get the sequence: $a_1, a_1 + \cfrac 1 a_2, a_1 + \cfrac 1 {a_2 + \cfrac 1 a_3} ... $

**Proprieties**  
1. They limit at the expanded number
2. They alternate being bigger or smaller than our expanded number

## How do you compute the continued fractions?

It seems that we can use the Euclidean algorithm to get convergents
https://math.stackexchange.com/questions/1469741/show-simple-continued-fraction-with-euclids-algorithm

## What are we looking for?

Good explanation: https://www.youtube.com/watch?v=OpPrrndyYNU

$N = pq \\
\varphi(N) = (p-1)(q-1) = pq - (p+q) + 1 = \overset {N >> p+q} {N - (p+q) + 1} \approx N  \ \ \ (1) \\
ed \equiv 1 \ mod \ \varphi(N) => ed = k \cdot \varphi(N) + 1; k\in \mathbb{Z} => \cfrac e {\varphi(N)} - \cfrac k d =\cfrac 1 {d\cdot \varphi(N)} {\approx 0} \overset{(1)} => \cfrac e N \approx \cfrac k d
$

*Idea:*  If i approximate $\frac e N$ with $\frac k d$ then my denominator is the decryption exponent => We want to find $\frac e N$

*Red flags:*
1. Since $ed \equiv 1 \ mod \ \varphi(N)$ and $\varphi(N)$ is even  => $d$ **must be odd**
2. $\varphi(N)$ is a whole number => $\cfrac {ed-1} k \in \mathbb{Z}$

Other test:
1. check $M^{ed} \equiv M \ mod \ N $ for some random $M$

## The quadratic equation

https://www.youtube.com/watch?v=ak0b-12bths&list=PLKXdxQAT3tCssgaWOy5vKXAR4WTPpRVYK&index=68 => we can use convergents to get a better approximate of an equation

$\varphi(N) = N - (p+q) + 1 \\ 
p + q = N - \varphi(N) + 1 $

Let $(x-p)(x-q) = 0$ be a quadratic equation with roots $p,q$  We have
* $(x-p)(x-q) = 0 <=> x^2 - (p+q)x + pq = 0 <=> x^2 - (N-\varphi(N) + 1)x + N = 0$ => if we have $\varphi(N)$ then we can find $p,q \in \mathbb{Z}$

So we have the next steps
1. We use the euclidean algorithm to get $\cfrac k d$ 
2. Check for red flags
3. Check quadratic equations to have whole solutions (find solutions with quadratic formula)

## Conditions for Wiener attack

If:
1. $q<p \leq 2q$
2. $d \leq \frac1 3 \cdot N^{\frac 1 4}$  

Then the Wiener attack will succeed

# Variant

https://eprint.iacr.org/2008/459.pdf

There is a variant of the Wiener attack that lets us find $d$ if it's a few bits longer than $N^{1/4}$

All rationals $\cfrac p q$ that satisfy:
$\left| \alpha - \cfrac p q \right| < \cfrac c q^2$ for a $c \in \mathbb{R}_+$ have the form $\cfrac p q = \cfrac {rp_{m+1} + s p_m } {rq_{m+1} + s q_m }$ for $0\leq r,s < 2c$ 

*idea* : we search $\cfrac k d$ between fractions of that form

So we want to get 
$\begin{align*}
k &= rp_{m+1} + s p_m \\
d &=  rq_{m+1} + s q_m
\end{align*}$
=> $\begin{align*}
r &= dp_m - kq_m \\
s &= dq_{m+1} - dqp_{m+1}
\end{align*}$

So we want to find the pair $(r,s)$
1. We can use exhaustive search if r and s are small
2. We can test $2^{e(rq_{m+1} + sq_m)} \equiv 2 \ mod N$


# Code

## Toy implementation

In [28]:
class WienerAttack:
    def __init__(self, e, N):
        self.e = e
        self.N = N
    
    
    def attack(self):
        e = self.e
        N = self.N
        a_list = []
        while True:
            quotient = e // N
            remainder = e % N
            e = N
            N = remainder
            a_list.append(quotient)
            #get convergent
            k, d = self.sequence_to_fraction(a_list)
            #assume its not the trivial solution
            if d == 1:
                continue
            #check red flags and conditions
            if self.checks(d, k):
                break          
        return d
            
    def attack_variant(self, r_max = 30, s_max = 30):
        e = self.e
        N = self.N
        a_list = []
        q0 = 1
        m_test = 2
        c_test = pow(m_test, e, N)
        while True:
            quotient = e // N
            remainder = e % N
            e = N
            N = remainder
            a_list.append(quotient)
            #get convergent
            k, q1 = self.sequence_to_fraction(a_list)
            if k == 0:
                continue
            #assume its not the trivial solution
            for r in range(r_max):
                for s in range(s_max):
                    d = r*q1 + s*q0
                    if d == 1:
                        continue
                    #check red flags and conditions
                    if pow(c_test, d, self.N) == m_test:
                        return d          
            q0 = q1
        return -1  
        
    def checks(self, d, k):
        #check parity
        if not d&1:
            return False
        #check if phi is whole
        if (e*d-1) % k != 0:
            return False
        else:
            phi = (e*d-1) // k
            
        #if doesnt pass the quadratic check return false
        if not self.check_quadratic(phi):
            return False
        return True
    
    
    def check_quadratic(self, phi):
        '''check if x^2 - (N-phi(N) + 1)x + N = 0 has integer solutions'''
        a = 1
        b = -(self.N - phi + 1)
        c = N
        
        delta = pow(b, 2) - 4 * a * c
        if delta < 0: # no real solutions
            return False
        #check if solutions are whole
        if not self.is_square(delta):
            return False
        else:
            delta_sqr = isqrt(delta)
        if  (-b + delta_sqr) % (2*a) != 0:
            return False
        else:
            x1 = (-b + delta_sqr) // (2*a)
        if  (-b - delta_sqr) % (2*a) != 0:
            return False
        else:
            x2 = (-b - delta_sqr) // (2*a)

        #check if solutions are good
        if x1 * x2 != self.N:
            return False
        return True
            
    def sequence_to_fraction(self, a_list):
        '''transform a continued fraction into a num/den fraction'''
        num, dem = 1, 0
        for a in reversed(a_list):
            num, dem = dem + num*a, num
        return num, dem
    
    def is_square(self, apositiveint):
        x = apositiveint // 2
        seen = set([x])
        while x * x != apositiveint:
            x = (x + (apositiveint // x)) // 2
            if x in seen: return False
            seen.add(x)
        return True

In [29]:
N = 64741
e = 42667
wa = WienerAttack(e, N)

In [30]:
d = wa.attack_variant()

In [31]:
d

3

**Bigger ints**

In [42]:
N = 0x8da7d2ec7bf9b322a539afb9962d4d2ebeb3e3d449d709b80a51dc680a14c87ffa863edfc7b5a2a542a0fa610febe2d967b58ae714c46a6eccb44cd5c90d1cf5e271224aa3367e5a13305f2744e2e56059b17bf520c95d521d34fdad3b0c12e7821a3169aa900c711e6923ca1a26c71fc5ac8a9ff8c878164e2434c724b68b508a030f86211c1307b6f90c0cd489a27fdc5e6190f6193447e0441a49edde165cf6074994ea260a21ea1fc7e2dfb038df437f02b9ddb7b5244a9620c8eca858865e83bab3413135e76a54ee718f4e431c29d3cb6e353a75d74f831bed2cc7bdce553f25b617b3bdd9ef901e249e43545c91b0cd8798b27804d61926e317a2b745
e = 0x86d357db4e1b60a2e9f9f25e2db15204c820b6e8d8d04d29db168c890bc8a6c1e31b9316c9680174e128515a00256b775a1a8ccca9c6936f1b4c2298c03032cda4dd8eca1145828d31466bf56bfcf0c6a8b4a1b2fb27de7a57fae7430048d7590734b2f05b6443ad60d89606802409d2fa4c6767ad42bffae01a8ef1364418362e133fa7b2770af64a68ad50ad8d2bd5cebb99ceb13368fb31a6e7503e753f8638e21a96af1b6498c18578ba89b98d70fa482ad137d28fe701b4b77baa25d5e84c81b26ee9bddf8cbb51a071c60dd57714de379cd4bc14932809ba18524a0a18e4133665cfc46e2c4fcfbc28e0a0957e5513a7307c422b87a6182d0b6a074b4d
m = bytes_to_long(b'secret')
c = encrypt_rsa(m, e, N)

In [43]:
wa = WienerAttack(e, N)
d = wa.attack()

In [36]:
wa = WienerAttack(e, N)
d = wa.attack_variant(r_max=5, s_max=5)

In [37]:
m_decr = pow(c, d, N)
print(long_to_bytes(m_decr))

b'secret'


In [45]:
#Parameters for the variant solution
N = 0x665166804cd78e8197073f65f58bca14e019982245fcc7cad74535e948a4e0258b2e919bf3720968a00e5240c5e1d6b8831d8fec300d969fccec6cce11dde826d3fbe0837194f2dc64194c78379440671563c6c75267f0286d779e6d91d3e9037c642a860a894d8c45b7ed564d341501cedf260d3019234f2964ccc6c56b6de8a4f66667e9672a03f6c29d95100cdf5cb363d66f2131823a953621680300ab3a2eb51c12999b6d4249dde499055584925399f3a8c7a4a5a21f095878e80bbc772f785d2cbf70a87c6b854eb566e1e1beb7d4ac6eb46023b3dc7fdf34529a40f5fc5797f9c15c54ed4cb018c072168e9c30ca3602e00ea4047d2e5686c6eb37b9
e = 0x2c998e57bc651fe4807443dbb3e794711ca22b473d7792a64b7a326538dc528a17c79c72e425bf29937e47b2d6f6330ee5c13bfd8564b50e49132d47befd0ee2e85f4bfe2c9452d62ef838d487c099b3d7c80f14e362b3d97ca4774f1e4e851d38a4a834b077ded3d40cd20ddc45d57581beaa7b4d299da9dec8a1f361c808637238fa368e07c7d08f5654c7b2f8a90d47857e9b9c0a81a46769f6307d5a4442707afb017959d9a681fa1dc8d97565e55f02df34b04a3d0a0bf98b7798d7084db4b3f6696fa139f83ada3dc70d0b4c57bf49f530dec938096071f9c4498fdef9641dfbfe516c985b27d1748cc6ce1a4beb1381fb165a3d14f61032e0f76f095d
m = bytes_to_long(b'secret')

In [53]:
c = encrypt_rsa(m, e, N)

In [52]:
wa = WienerAttack(e, N)
d = wa.attack_variant(r_max = 5, s_max=5)

In [54]:
long_to_bytes(pow(c, d, N))

b'secret'

## Library

https://github.com/orisano/owiener

In [116]:
import owiener
d = owiener.attack(e, N)
m_decr = pow(c, d, N)
print(long_to_bytes(m_decr))

b'secret'


# Resources

* Wiki: https://en.wikipedia.org/wiki/Wiener%27s_attack
* Paper: http://monge.univ-mlv.fr/~jyt/Crypto/4/10.1.1.92.5261.pdf